### Search Engine with Tools & Agents


* Tools -> are interfaces that an agent, chain, or LLM can use to interact with the world.

* Agents -> The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.


#### links

* Tools -> https://python.langchain.com/v0.1/docs/modules/tools/
* Agents ->https://python.langchain.com/v0.1/docs/modules/agents/

In [3]:
## Arxiv -- Research

## Tools Creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki_tool.name

'wikipedia'

In [5]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv_tool.name)

arxiv


In [6]:
## Combine the tools

tools = [wiki_tool, arxiv_tool]

In [ ]:
## Custom tool
## RAG Tool

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
loader  = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = vectordb.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000207F4B9CCD0>, search_kwargs={})

In [9]:
## now I have to convert this retriever into a tool
from langchain.tools.retriever import create_retriever_tool

##  create_retriever_tool (retriever, tool-name, description/info)
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")

retriever_tool.name

'langsmith-search'

In [10]:
## combine all the tools

tools = [wiki_tool, arxiv_tool, retriever_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\KIIT\\OneDrive\\Desktop\\Langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000207E4063D90>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000

In [12]:
## Run all the Tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()

import os
groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key= groq_api_key, model_name="Llama3-8b-8192")

In [13]:
## prompt template
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
## Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [15]:
## Agent Executor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools= tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_executor.invoke({"input": "Tell me about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!
1. Install LangSmith​
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key​
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment​
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necess

{'input': 'Tell me about LangSmith',
 'output': "I've got the information about LangSmith. It seems to be a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, making it easier to ship quickly and with confidence.\n\nTo get started with LangSmith, you need to install it using pip or yarn, create an API key, set up your environment, and log your first trace. LangSmith provides an API key that can be used to access the LangSmith experiments UI and explore the results of your evaluation.\n\nLangSmith also integrates with LangChain, an open-source framework for building LLM applications. This allows developers to use LangSmith without having to write custom code for tracing and logging.\n\nIt seems like LangSmith is a powerful tool for building and deploying LLM applications, and it's great that you're interested in learning more about it!"}

In [17]:
agent_executor.invoke({"input": "what is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Machine learning#Applications'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Machine learning#Applications'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks w

{'input': 'what is machine learning',
 'output': "The tool calls did not yield any new information. It appears that the tool calls have already provided the necessary information to answer the user's question about machine learning. Therefore, I will provide a direct response:\n\nMachine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks with some degree of autonomy."}

In [18]:
agent_executor.invoke({"input": "What is this paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `langsmith-search` with `{'query': 'Attention Is All You Need'}`


5. Run your first evaluation​
Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.

Get started with LangSmith | 🦜️🛠️ LangSmith

Click the link printed out by your evaluation run to access the LangSmith experiments UI,
and explore the results of your evaluation.
Learn more about evaluation in the tutorials, conceptual guide, and how-to guides.
Was this page helpful?You can leave detailed feedback on GitHub.NextConcepts1. Install LangSmith2. Create an API key3. Set up you

{'input': 'What is this paper 1706.03762 about?',
 'output': 'Since the tool calls have yielded the desired information, I can provide a direct response. The paper "Attention Is All You Need" by Vaswani et al. (2017) introduces the Transformer model, a type of neural network architecture that uses self-attention mechanisms to process input sequences. The paper presents a new approach to sequence transduction that does not rely on traditional recurrent neural network (RNN) or convolutional neural network (CNN) architectures. Instead, it uses a self-attention mechanism to weigh the importance of different input elements for each position in the output sequence. This allows the model to attend to different parts of the input sequence simultaneously, which can be beneficial for tasks such as machine translation, text summarization, and language modeling.'}